# The Expedia Kaggle competition  

The Expedia competition challenges you with predicting what hotel a user will book based on some attributes about the search user is conducting on Expedia. Before we dive into any coding ,we'll need to put in time to understand both the problem and the data.  

## A quick glance at the columns  
The first step is to look at the description of the columns of the dataset. You can find it here[https://www.kaggle.com/c/expedia-hotel-recommendations].  Towards the bottom of the page,you'll see a description of each column in the data.```destinations.csv```  contains information about the regions users search in for hotels.We wont worry about what we're predicting just yet,we'' focus on understanding the columns.  

## Expedia  
Since the competitions consists of even data from users booking hotels on Expedia,we'll need to spend some time understanding the Expedia site.  
Looking at the booking flow will help us contextualize the fileds in the data, and how they tie into using Expedia.    

## Exploring the data with Pandas  


In [2]:
import pandas  as pd

destinations =pd.read_csv("./data/destinations.csv")  
test = pd.read_csv("./data/test.csv")  
train =pd.read_csv("./data/train.csv") 

In [3]:
train.shape


(37670293, 24)

In [4]:
test.shape

(2528243, 22)

We have about `37` million training set rows , and `2` million testing set rows,which will make this problem a bit challenging to work with.

In [5]:
train.head(5)

,date_time,site_name,posa_continent,user_location_country,user_location_region,user_location_city,orig_destination_distance,user_id,is_mobile,is_package,...,srch_children_cnt,srch_rm_cnt,srch_destination_id,srch_destination_type_id,is_booking,cnt,hotel_continent,hotel_country,hotel_market,hotel_cluster
0,2014-08-11 07:46:59,2,3,66,348,48862,2234.2641,12,0,1,...,0,1,8250,1,0,3,2,50,628,1
1,2014-08-11 08:22:12,2,3,66,348,48862,2234.2641,12,0,1,...,0,1,8250,1,1,1,2,50,628,1
2,2014-08-11 08:24:33,2,3,66,348,48862,2234.2641,12,0,0,...,0,1,8250,1,0,1,2,50,628,1
3,2014-08-09 18:05:16,2,3,66,442,35390,913.1932,93,0,0,...,0,1,14984,1,0,1,2,50,1457,80
4,2014-08-09 18:08:18,2,3,66,442,35390,913.6259,93,0,0,...,0,1,14984,1,0,1,2,50,1457,21


There are few things that immediately stick out:  
- ```date_time``` could be useful in our predictions,so we'll need to convert it.  
- Most of the columns are integers or floats ,so we can't do a lot of feature engineering.For example,```user_location_country``` isn't the name of a country ,it's an integer.This makes it harder to create new features,
because we don't know exactly which value means

In [6]:
test.head(5)

,id,date_time,site_name,posa_continent,user_location_country,user_location_region,user_location_city,orig_destination_distance,user_id,is_mobile,...,srch_ci,srch_co,srch_adults_cnt,srch_children_cnt,srch_rm_cnt,srch_destination_id,srch_destination_type_id,hotel_continent,hotel_country,hotel_market
0,0,2015-09-03 17:09:54,2,3,66,174,37449,5539.0567,1,1,...,2016-05-19,2016-05-23,2,0,1,12243,6,6,204,27
1,1,2015-09-24 17:38:35,2,3,66,174,37449,5873.2923,1,1,...,2016-05-12,2016-05-15,2,0,1,14474,7,6,204,1540
2,2,2015-06-07 15:53:02,2,3,66,142,17440,3975.9776,20,0,...,2015-07-26,2015-07-27,4,0,1,11353,1,2,50,699
3,3,2015-09-14 14:49:10,2,3,66,258,34156,1508.5975,28,0,...,2015-09-14,2015-09-16,2,0,1,8250,1,2,50,628
4,4,2015-07-17 09:32:04,2,3,66,467,36345,66.7913,50,0,...,2015-07-22,2015-07-23,2,0,1,11812,1,2,50,538


- It looks like all the dates in test.csv are later than the dates in ```train.csv``` .The testing set contains dates from 2015 ,and the training set contains dates from 2013 and 2014.  

- It looks like the user ids in ```test.csv``` are a subset of the user ids in ```train.csv``` , given the overlapping integer ranges.We can confirm this later on.  

- The ```is_booking``` column always looks to be 1 in `test.csv`  

# Figuring out what to predict  
## What we're predicting  

We'll be predicting which `hotel_cluster` a user will book after a given search.According to the description, there are 100 clusters in total.  

## How we'll be scored
The evaluation page says that we'll be scored using Mean Average Precision @5,which means that we'll need to make 5 cluster predictions for each row,and will be scored on whether or not the correct predictions appear in our list.If the correct prediction comes earlier in the list,we get more points.  

For example,if the "correct" cluster is 3, and we predict 4,43,60,3,20 our score 'll be lower than if we predict 3,4,43,60,20.We should put predictions we're more certain about earlier in our list of predictions.  


# Exploring hotel clusters

Now that we know what we're predicting,it's time to dive in and explore `hotel_cluster`.We can use the `value_counts` method on Series to do this:

In [7]:
train["hotel_cluster"].value_counts()

91    1043720
41     772743
48     754033
64     704734
65     670960
5      620194
98     589178
59     570291
42     551605
21     550092
70     545572
18     545284
83     534132
46     534038
25     530591
62     518809
95     509266
28     507016
68     503797
82     503755
37     496061
50     489892
30     489287
9      488328
58     483253
97     479446
16     477868
72     457463
1      452694
99     444887
       ...   
19     282893
84     278264
66     273505
38     269246
87     260398
23     259233
12     259022
31     257587
67     255946
43     253578
7      252447
54     250745
92     244343
89     243560
45     241408
49     240124
3      225250
80     220218
60     217919
71     216054
93     214293
86     209054
14     192299
75     165226
24     164127
35     139122
53     134812
88     107784
27     105040
74      48355
Name: hotel_cluster, dtype: int64

The output above is truncated,but it shows that the number of hotels in each cluster is fairly evenly distributed.There doesn't appear to be any relationship betweeb cluster number and the number of items.  

# Exploring train and test user ids  

Finally,we'll confirm our hypothesis that all the test user ids are found in the `Train` DataFrame.We can do this by finding the unique values for `user_id` in `test`,and seeing if they all exist in `train`.   

In the code below,we'll: 

- Create a set of all the unique `test` user ids 
- Create a set of all the unnique `train` user ids 
- Figure out how many `test` user ids are in the `train` user ids  
- See if the count matches the total number of `test` user ids.  


In [8]:
test_ids = set(test.user_id.unique())  
train_ids = set(train.user_id.unique())  
intersection_count = len(test_ids & train_ids)  
intersection_count == len(test_ids) 


True

Looks like our hypothesis is correct,which will make working with this data much easier !

# Downsampling our Kaggle data  
 The entire `train.csv` dataset contains 37 million rows,which makes it hard to experiment with dfferent techniques. Ideally,we want a small enough dataset that lets us quickly iterate through different approaches but is still representative of the whole training data.  
 
 We can do this by first randomly sampling rows from our data , then 
 selecting new training and testing datasets from `train.csv`.By selecting both sets from `train.csv` , we'll have the true `hotel_cluster` label for every row, and we'll be able to calculate our accuracy as we test techniques.  

## Add in times and dates
The first step is to add `month` and `year` fields to train.Because the `train` and `test` data is differetiated by date,we'll need to add date fields to allow us to segment our data into two sets the same way.If we add `year` and `month` fields, we can split our data into training and testing sets using them.  

The code below will:  

- convert the `date_time` column in `train` from an object to a `datetime` value.This makes it easier to work with as a date.  

- Extract the `year` and `month` from `date_time`,and assign them to their own columns.  


In [9]:
train["date_time"] = pd.to_datetime(train["date_time"]) 
train["year"] = train["date_time"].dt.year  
train["month"] = train["date_time"].dt.month  

# Pick 10000 users  
Because the user ids in `test` are a subset of the user ids in `train` , we'll need to do our random sampling in a way that preserves the full data of each user.We can accomplish this by selecting a certain number of users randomly, then only picking rows from `train` where `user_id` is in our random sample of user ids.  

In [10]:
import random 

unique_users = train.user_id.unique()  

sel_user_ids = [unique_users[i] for i in sorted(random.sample(range(len(unique_users)),10000))]
sel_train = train[train.user_id.isin(sel_user_ids)] 

The above codes created a DataFrame called `sel_train` that only contains data from `10000` users.  

The above codes creates a DataFrame called `sel_train` that only contains data from 10000 users

# Pick new training and testing sets  

We'll now need to pick new training and testting sets from `sel_train` .We'll call these sets `t1` and `t2`.  



In [11]:
t1 =sel_train[((sel_train.year == 2013) | ((sel_train.year == 2014) & (sel_train.month < 8)))]
t2=sel_train[((sel_train.year == 2014) & (sel_train.month >= 8))] 

In the original `train` and `test` DataFrames,`test` contained data from `2015` and `train` contained data from `2013` and `2014`.We split this data so that anything after `July 2014` is in `t2`,and anything before is in `t1`. This gives us smaller training and testing sets with similar characteristics to `train` and `test`.  

# Remove click events  
If `is_booking` is 0,it represents a click, and a `1` represents a booking.`test` contains only booking events,so we'll need to sample `t2` to only contain bookings as well.  

In [12]:
t2=t2[t2.is_booking == True]  

# A simple algorithm


The most simple technique we could try on this data is to find the most common clusters across the data,then use them as predictions.  


We can again use the value_counts method ot help us here: 

In [13]:
most_common_clusters=list(train.hotel_cluster.value_counts().head().index)

In [14]:
most_common_clusters 

[91, 41, 48, 64, 65]

The above code will give us a list of the 5 most common clusters in `train`. This is because the head method returns the first 5 rows by default ,and the index property will return the index of the DataFrame,which is the hotel_cluster after running the `value_counts` method.

# Generating predictions  

We can turn `most_common_clusters` into a list of predictions by making the same prediction for each row.  



In [15]:
predictions =[most_common_clusters for i in range(t2.shape[0])] 

This will create a list with as many elements as there are rows in `t2`. Each element will equal to `most_common_clusters`

# Evaluating error 
In order to evaluate error ,we'll first need to figure out how to compute Mean Average Precision

In [16]:
import ml_metrics as metrics  
target = [[l] for l in t2["hotel_cluster"]] 
metrics.mapk(target,predictions,k=5) 

0.064016334374249348

Our result here isn't great,but we've just generated our first set of predictions ,and evaluated our error! The framework we've built will allow us to quickly test out a variety of techniqes and see how they score.We're well on our way to building a good -performing solution for the leaderboard.  

# Finding correlations  

Before we move on to creating a better algorithm , let's see if anything correlates well with `hotel_cluster` .This will tell us if we should dive more into any particular columns.  

We can linear correations in the training set using the `corr` method:  



In [17]:
train.corr()["hotel_cluster"] 

site_name                   -0.022408
posa_continent               0.014938
user_location_country       -0.010477
user_location_region         0.007453
user_location_city           0.000831
orig_destination_distance    0.007260
user_id                      0.001052
is_mobile                    0.008412
is_package                   0.038733
channel                      0.000707
srch_adults_cnt              0.012309
srch_children_cnt            0.016261
srch_rm_cnt                 -0.005954
srch_destination_id         -0.011712
srch_destination_type_id    -0.032850
is_booking                  -0.021548
cnt                          0.002944
hotel_continent             -0.013963
hotel_country               -0.024289
hotel_market                 0.034205
hotel_cluster                1.000000
year                        -0.001050
month                       -0.000560
Name: hotel_cluster, dtype: float64

This tell us that no column correlate linearly with `hotel_cluster`.This makes sense ,because there is no linear ordering to `hotel_cluster`.For example, having a higher cluster number isn't tied to having a higher `srch_destnation_id`.  

Unfortunately, this means that techniques like linear regression and logistic regression won't work well on our data,because they rely on linear correlations between predictors and targets.

# Creating better predictions for our Kaggle entry

This data for this competition is quite difficult to make predictions on using machine learning for a few reasons:  

- There are millions of rows, which increases runtime and memory usage for algorithms.  
- There are `100` different clusters. and according to the competition admins,the boundaries are fairly fuzzy,so it will likely be hard to make predictions.As the number of cluster increases,classifiers generally decrease in accuracy.  
- Nothing is linearly correlated with the target(`hotel_clusters`),meaning we can't use fast machine learning techniques like linear regression.  

For these reasons,machine learning probably wont work well on our data,but we can try an algorithm and find out.  

# Generating features  

The first step in applying machine learning is to generate features.We can generate features using both what's availaible in the training data, and what's availaible in `destinations` .We haven't looked at `destinations` yet , so let's take a quick peek.   

# Generating features from destinations  

Destinations contains an id that corresponds to `srch_destination_id`,along with `149` columns of latent information about that destination.Here's a sample

In [18]:
destinations.head()

,srch_destination_id,d1,d2,d3,d4,d5,d6,d7,d8,d9,...,d140,d141,d142,d143,d144,d145,d146,d147,d148,d149
0,0,-2.198657,-2.198657,-2.198657,-2.198657,-2.198657,-1.897627,-2.198657,-2.198657,-1.897627,...,-2.198657,-2.198657,-2.198657,-2.198657,-2.198657,-2.198657,-2.198657,-2.198657,-2.198657,-2.198657
1,1,-2.181690,-2.181690,-2.181690,-2.082564,-2.181690,-2.165028,-2.181690,-2.181690,-2.031597,...,-2.165028,-2.181690,-2.165028,-2.181690,-2.181690,-2.165028,-2.181690,-2.181690,-2.181690,-2.181690
2,2,-2.183490,-2.224164,-2.224164,-2.189562,-2.105819,-2.075407,-2.224164,-2.118483,-2.140393,...,-2.224164,-2.224164,-2.196379,-2.224164,-2.192009,-2.224164,-2.224164,-2.224164,-2.224164,-2.057548
3,3,-2.177409,-2.177409,-2.177409,-2.177409,-2.177409,-2.115485,-2.177409,-2.177409,-2.177409,...,-2.161081,-2.177409,-2.177409,-2.177409,-2.177409,-2.177409,-2.177409,-2.177409,-2.177409,-2.177409
4,4,-2.189562,-2.187783,-2.194008,-2.171153,-2.152303,-2.056618,-2.194008,-2.194008,-2.145911,...,-2.187356,-2.194008,-2.191779,-2.194008,-2.194008,-2.185161,-2.194008,-2.194008,-2.194008,-2.188037



THe competition doesn't tell us exactly what each latent feature is,but it's safe to assume that it's some combination of destination characteristics,like name,description and more.These latent features were converted to numbers, so they could be anonymized.  

We can use the destination information as features in a machine learning alogrithm, but we'll need to compress the number of columns down first,to minimize runtime.We can use PCA to do this. PCA will reduce the number of columns in a matrix while trying to preserve the same amount of variance per row.Ideally,PCA will compress all the information contained in all the columns into less,but in practice ,some information is lost. 

In the code below :
- Initialize a PCA model using scikit-learn  
- Specify that we want to only have `3` columns in our data
- Transform the columns `d1-d149` into `3` columns 

In [19]:
from sklearn.decomposition import PCA  

pca = PCA(n_components=3)  
dest_small = pca.fit_transform(destinations[["d{0}".format(i+1) for i in range(149)]])
dest_small = pd.DataFrame(dest_small)  
dest_small["srch_destination_id"]= destinations["srch_destination_id"] 

The above code compresses the `149` columns in destinations down to `3` columns, and creates a new DataFrame called `dest_small`.We preserve most of the variance in `destinations` while doing this,so we don't lose a lot of information , but save a lot of runtime for a machine learning algorithm. 

# Generating features  
Now that the preliminaries are done with, we can generate our features. We'll do the following:  
- Generate new date features based on `date_time` ,`srch_ci`, and `srch_co`.  
- Remove non-numeric columns like `date-time`.  
- Add in features from dest_small   
- Replace any missing values with -1  

In [20]:

def calc_fast_features(df):
    df["date_time"] = pd.to_datetime(df["date_time"])
    df["srch_ci"] = pd.to_datetime(df["srch_ci"], format='%Y-%m-%d', errors="coerce")
    df["srch_co"] = pd.to_datetime(df["srch_co"], format='%Y-%m-%d', errors="coerce")
    
    props = {}
    for prop in ["month", "day", "hour", "minute", "dayofweek", "quarter"]:
        props[prop] = getattr(df["date_time"].dt, prop)
    
    carryover = [p for p in df.columns if p not in ["date_time", "srch_ci", "srch_co"]]
    for prop in carryover:
        props[prop] = df[prop]
    
    date_props = ["month", "day", "dayofweek", "quarter"]
    for prop in date_props:
        props["ci_{0}".format(prop)] = getattr(df["srch_ci"].dt, prop)
        props["co_{0}".format(prop)] = getattr(df["srch_co"].dt, prop)
    props["stay_span"] = (df["srch_co"] - df["srch_ci"]).astype('timedelta64[h]')
        
    ret = pd.DataFrame(props)
    
    ret = ret.join(dest_small, on="srch_destination_id", how='left', rsuffix="dest")
    ret = ret.drop("srch_destination_iddest", axis=1)
    return ret

df = calc_fast_features(t1)

df.fillna(-1, inplace=True)

/Users/KrishMahajan/anaconda/lib/python3.5/site-packages/ipykernel/__main__.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  app.launch_new_instance()
/Users/KrishMahajan/anaconda/lib/python3.5/site-packages/ipykernel/__main__.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/Users/KrishMahajan/anaconda/lib/python3.5/site-packages/ipykernel/__main__.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentatio

In [21]:
df.head()

,channel,ci_day,ci_dayofweek,ci_month,ci_quarter,cnt,co_day,co_dayofweek,co_month,co_quarter,...,srch_rm_cnt,stay_span,user_id,user_location_city,user_location_country,user_location_region,year,0,1,2
878,2,4.0,5.0,10.0,4.0,1,6.0,0.0,10.0,4.0,...,1,48.0,5776,42328,194,38,2014,0.557325,0.02909,0.030218
879,2,4.0,5.0,10.0,4.0,1,6.0,0.0,10.0,4.0,...,1,48.0,5776,42328,194,38,2014,0.557325,0.02909,0.030218
880,2,4.0,5.0,10.0,4.0,1,6.0,0.0,10.0,4.0,...,1,48.0,5776,42328,194,38,2014,0.557325,0.02909,0.030218
881,2,4.0,5.0,10.0,4.0,1,6.0,0.0,10.0,4.0,...,1,48.0,5776,42328,194,38,2014,-0.218982,0.24742,0.123216
882,2,4.0,5.0,10.0,4.0,1,6.0,0.0,10.0,4.0,...,1,48.0,5776,42328,194,38,2014,-0.218982,0.24742,0.123216


The above will calculate features such as length of stay,check in day, and check out month. These features will help us train a machine learning algorithm later on.  

Replaacing missing values with `-1` isn't the best choice,but it will work fine for now, and we can always optimize the behavior later on.

# Machine Learning

Now that we have features for our training data.we can try machine learning.We'll use `3-fold cross validation` across the training set to generate a reliable error estimate. Cross validation splits the training set up into `3` parts, then predicts `hotel_cluster` for each part using the other parts to train with   

we'll generate predictions using the Random Forest algorithm.Random forests build trees,which can fit to nonlinear tendencies in data.This will enable us to make predictions,even though none of our columns are linearly related.  

We'll first initialize the model and compute cross validation scores:

In [22]:
predictors = [c for c in df.columns if c not in ["hotel_cluster"]] 

from sklearn import cross_validation  
from sklearn.ensemble import RandomForestClassifier    

clf = RandomForestClassifier(n_estimators=10,min_weight_fraction_leaf=0.1) 
scores=cross_validation.cross_val_score(clf,df[predictors],df['hotel_cluster'],cv=3)

In [23]:
scores

array([ 0.06628094,  0.07263112,  0.06571247])

The above code doesn't give us very good accuracy , and confirms our original suspicion that machine learning isn't a great approach to this problem. However,classifiers tend to have lower accuracy when there is a high cluster count.We can instead try training `100` binary classifiers.Each classifier will just determine if a row is in it's cluster,or not.This will entail training one classifier per label in `hotel_cluster`

# Binary classifiers  

We'll try again Random Forests,but each forest will predict only a single hotel cluster.We'll use `2` fold cross validation for speed,and only train `10` trees per label.  

In the code below, we:  

- Loop across each unique `hotel_cluster`  
  - Train a Random Forest classifier using 2-fold cross validation 
  - Extract the probabilities from the classifier that row is in the unique `hotel_cluster`   
  
- Combine all the probabilities  
- For each row, find the `5` largest probabilities , and assign those `hotel_cluster` values as predictions  
- Compute accuracy using `mapk`.

In [24]:
from sklearn.ensemble import RandomForestClassifier  
from sklearn.cross_validation import KFold  
from itertools import chain  

all_probs = [] 
unique_clusters = df["hotel_cluster"].unique()  

for cluster in unique_clusters:
    df["target"]=1
    df['target'][df["hotel_cluster"]!=cluster]=0  
    predictors = [col for col in df if col not in ['hotel_cluster','target']]
    probs = []  
    cv=KFold(len(df["target"]),n_folds=2) 
    clf = RandomForestClassifier(n_estimators=10,min_weight_fraction_leaf=0.1)
    for i,(tr,te) in enumerate(cv):
        clf.fit(df[predictors].iloc[tr],df["target"].iloc[tr])
        preds = clf.predict_proba(df[predictors].iloc[te]) 
        probs.append([p[1] for p in preds])
    full_probs = chain.from_iterable(probs)
    all_probs.append(list(full_probs))  
    
prediction_frame = pd.DataFrame(all_probs).T  
prediction_frame.columns = unique_clusters  

/Users/KrishMahajan/anaconda/lib/python3.5/site-packages/ipykernel/__main__.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/Users/KrishMahajan/anaconda/lib/python3.5/site-packages/ipykernel/__main__.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/Users/KrishMahajan/anaconda/lib/python3.5/site-packages/ipykernel/__main__.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/Users/KrishMahajan/anaconda/lib/python3.5/site-packages/ipykernel/__main__.py:10: SettingWithCopyWarning: 
A

In [25]:
prediction_frame.head()

,78,30,44,58,60,46,52,51,11,55,...,62,86,22,66,27,63,35,49,14,74
0,0.016444,0.024289,0.010145,0.015385,0.010401,0.033823,0.001555,0.008326,0.009974,0.012757,...,0.028591,0.006966,0.010353,0.001734,0.000892,0.012435,0.004348,0.003283,0.005536,0.000010
1,0.016444,0.024289,0.010145,0.015385,0.010401,0.033823,0.001555,0.008326,0.009974,0.012757,...,0.028591,0.006966,0.010353,0.001734,0.000892,0.012435,0.004348,0.003283,0.005536,0.000010
2,0.016444,0.024289,0.010145,0.015385,0.010401,0.033823,0.001555,0.008326,0.009974,0.012757,...,0.028591,0.006966,0.010353,0.001734,0.000892,0.012435,0.004348,0.003283,0.005536,0.000010
3,0.015720,0.019993,0.013153,0.014921,0.009270,0.038614,0.004398,0.009378,0.010405,0.006474,...,0.027113,0.004772,0.012836,0.001690,0.000712,0.012422,0.004499,0.005670,0.006478,0.001101
4,0.015720,0.021975,0.013153,0.014921,0.009270,0.037998,0.004398,0.008838,0.010405,0.006474,...,0.027113,0.004772,0.012836,0.001690,0.000712,0.012422,0.004499,0.005670,0.005940,0.001101


Basically now for every observation we calculated their probability for being in every cluster.

In [26]:
def find_top_5(row):
    return list(row.nlargest(5).index)

preds = []
for index, row in prediction_frame.iterrows():
    preds.append(find_top_5(row))

metrics.mapk([[l] for l in t2["hotel_cluster"]], preds, k=5)

0.047579870285851547

Accuracy here is worse than before,and people on the leaderboard have much better accuracy scores.We'll need to abandon machine learning and move to the next technique in order to compete.Machine learning can be a powerful technique,but it isn't always the right approach to every problem

# Top clusters based on hotel_cluster for each region  

There are a few `Kaggle Scripts`  for the competition that involve aggregating `hotel_cluster` based on `orig_destination_distance`, or `srch_destination_id`.Aggregating on `srch_destination_id` will find the most popular hotel clusters for each destination.We'll be then able to predict that a user who searches for a destination is going to one of the most popular hotel clusters for that destination. Think of this as a more granular version of the most common clusters technique we used earlier.   

We can first generate scores for each `hotel_cluster` in each `srch_destinaation_id` .We'll weight bookings higher than clicks.This is because the test date is all booking data , and this is what we want to predict. We want to include click information,but downweight it to reflect this. Step by step,we'll:  

- Group `t1` by `srch_destination_id` and `hotel_cluster`  
- Iterate through each group ,and:  
  - Assign `1` point to each hotel cluster where `is_booking` is true. 
  - Assign `.15` points to each hotel cluster where `is_booking` is false.
  - Assign the score to the `srch_destination_id`/ `hotel_cluster` 
    combination in a dictionary  

  
Here's the code to accomplish the above steps:   
 

In [27]:
def make_key(items):  
    return "-".join([str(i) for i in items])  

In [57]:
match_cols = ["srch_destination_id"]  
clusters_cols = match_cols + ['hotel_cluster']
#groups = t1.groupby(clusters_cols)  # For test submission
groups = train.groupby(clusters_cols)# For final submission

In [59]:
top_clusters = {}  
for name,group in groups:  
    clicks = len(group.is_booking[group.is_booking == False])
    bookings = len(group.is_booking[group.is_booking == True]) 
    score = bookings + .15*clicks  
    clus_name = make_key(name[:len(match_cols)])
    #print(clus_name)
    if clus_name not in top_clusters:
        top_clusters[clus_name] = {} 
    top_clusters[clus_name][name[-1]] = score

In [43]:
top_clusters

{'2439': {93: 0.3},
 '12249': {5: 0.44999999999999996,
  10: 0.15,
  11: 0.15,
  14: 0.15,
  16: 0.44999999999999996,
  29: 0.15,
  30: 0.44999999999999996,
  35: 0.15,
  61: 0.15,
  73: 0.15,
  77: 0.75,
  81: 0.15,
  82: 0.3,
  91: 0.15},
 '43240': {31: 0.3, 38: 0.3, 58: 0.3, 90: 0.15, 99: 0.15},
 '23425': {91: 1.3},
 '22213': {57: 0.3, 61: 0.15, 82: 0.15},
 '63480': {56: 1.3},
 '18790': {3: 0.15,
  4: 1.2,
  8: 0.3,
  10: 1.75,
  13: 2.05,
  14: 0.15,
  15: 0.75,
  16: 0.3,
  18: 0.3,
  20: 0.3,
  23: 0.3,
  28: 1.3499999999999999,
  31: 0.15,
  32: 0.3,
  37: 4.0,
  39: 3.65,
  40: 0.3,
  41: 1.7999999999999998,
  47: 7.25,
  48: 1.3499999999999999,
  50: 5.5,
  60: 0.3,
  72: 1.6,
  76: 1.3,
  77: 0.44999999999999996,
  83: 0.75,
  90: 1.5,
  91: 4.65,
  93: 0.6,
  94: 0.15,
  96: 0.15},
 '33452': {62: 1.6, 78: 1.6},
 '12213': {2: 2.3499999999999996,
  5: 0.44999999999999996,
  13: 0.75,
  25: 3.1,
  28: 2.2,
  31: 0.3,
  38: 0.3,
  44: 0.15,
  46: 0.3,
  58: 0.6,
  59: 0.15,
  64

we'll next want to transform this dictionary to find the top 5 hotel clusters for each `srch_destination_id`.We'll next want to transform this dictionary to find the top 5 hotel clusters for each `srch_destination_id`.In order to this,we'll:  

- Loop through each key in `top_clusters`
- Find the top `5` clusters for that key
- Assign the top `5` clusters to a new dictionary ,`cluster_dict`  

Here's the code:

In [60]:
import operator  
cluster_dict = {}  

for n in top_clusters:
    tc = top_clusters[n] 
    top = [l[0] for l in sorted(tc.items(),key=operator.itemgetter(1),reverse=True)[:5]]
    cluster_dict[n]=top

In [48]:
cluster_dict

{'2439': [93],
 '12249': [77, 5, 16, 30, 82],
 '43240': [58, 38, 31, 99, 90],
 '23425': [91],
 '22213': [57, 82, 61],
 '63480': [56],
 '18790': [47, 50, 91, 37, 39],
 '33452': [78, 62],
 '12213': [98, 25, 2, 97, 28],
 '21645': [42, 48, 13, 89, 41],
 '19037': [30],
 '34208': [22],
 '17898': [33],
 '45194': [77],
 '431': [61],
 '59': [38, 75, 64, 46, 61],
 '54847': [62, 20],
 '53631': [33],
 '9648': [99, 26],
 '375': [82, 36, 26, 22, 78],
 '7667': [72, 91],
 '21469': [58, 67, 80, 3, 36],
 '51285': [85],
 '26100': [47, 48],
 '24404': [72],
 '17813': [2, 72, 47, 68, 70],
 '24235': [67],
 '41615': [73],
 '3530': [32, 42],
 '63922': [32],
 '3331': [51, 42],
 '35798': [9, 59],
 '52746': [5],
 '20968': [82, 43],
 '19372': [5, 25, 37, 41, 99],
 '31109': [39, 48, 91, 94],
 '26285': [35],
 '7982': [97],
 '41398': [63],
 '8279': [98, 70, 41, 19, 83],
 '33401': [48, 11, 22, 78, 14],
 '17933': [46],
 '17841': [28],
 '12748': [6, 68, 76, 9, 16],
 '27211': [22, 25, 67, 78, 58],
 '8244': [91, 48, 42, 7

# Making predictions based on destination

Once we know the top clusters for each `srch_destination_id` ,we can quickly make predictions .To make predictions,all we have to do is:  

- Iterate through each row in `t2`  
  - Extract the `srch_destination_id` for the row 
  - Find the top clusters for that destination id
  - Append the top clusters to preds
  
Here's the code:

In [61]:
preds = [] 
for index,row in t2.iterrows():
    key= make_key(row[m] for m in match_cols)  
    if key in cluster_dict:
        preds.append(cluster_dict[key]) 
    else:
        preds.append([])  

At the end of the loop,`preds` will be a list of lists containing our predictions.  
It will lool loke this:  

In [51]:
preds

[[2, 30, 82, 90, 58],
 [98, 56, 55, 70, 41],
 [2, 51, 18, 48, 91],
 [54, 79, 50, 47],
 [39],
 [39],
 [39],
 [99, 42, 58, 78, 82],
 [95, 55, 98, 19, 68],
 [1, 79, 45, 24, 54],
 [13, 51, 28, 40, 50],
 [42, 28, 48, 60, 76],
 [76, 68, 83, 91, 77],
 [49, 4, 55, 6, 16],
 [98, 21, 95, 68, 9],
 [98, 21, 95, 68, 9],
 [51, 40, 83],
 [51, 40, 83],
 [26, 12, 67, 46, 81],
 [94, 95, 68, 50, 32],
 [76, 47, 39, 14, 7],
 [],
 [64, 82, 25, 5, 30],
 [2, 97, 11, 25, 64],
 [9, 98, 95, 49, 64],
 [59, 89, 98, 9, 21],
 [79, 89, 39, 21, 0],
 [79, 89, 39, 21, 0],
 [98, 21, 97, 25, 64],
 [64, 29, 59, 2, 46],
 [82, 29, 99, 64, 36],
 [70, 95, 18, 83, 68],
 [65, 87, 52, 66, 5],
 [70, 95, 18, 83, 68],
 [70, 91, 41, 98, 68],
 [66, 65, 87, 8, 62],
 [65, 18, 50, 52, 87],
 [],
 [19, 21, 72, 98, 41],
 [95, 91, 18, 68, 59],
 [1, 45, 79, 54, 24],
 [19, 21, 72, 98, 41],
 [55, 72, 25, 68, 59],
 [25, 38, 2, 68, 5],
 [36, 64, 46, 15, 9],
 [25, 38, 2, 68, 5],
 [22, 97, 37, 86, 33],
 [25, 38, 2, 68, 5],
 [50, 42, 16, 81, 2],
 [1

# Calculating error  
once we have our predictions,we can compute our accuracy using the `mapk` 
function from earlier:  

In [52]:
metrics.mapk([[l] for l in t2["hotel_cluster"]],preds,k=5) 

0.2429277764432701

We're doing pretty well ! We boosted our accuracy 4x over the best machine learning approach , and we did it with a far faster and simpler approach  

YOu have noticed that this value is quite a bit lower than accuracies on the leaderboard.Local testing results in a lower accuracy value than submitting,so this approach will actually do fairly well on the leadeboard.Differences in leaderboard score and local score can come down to a few factors:  

- Different data locally and in the hidden set that leaderboard scores are computed on. FOr example,we're computing error in a sample of the training set, and the leaderboard score is computed on the testing set.  

- Techniques that result in higher accuracy with more training data.We're only using a small subset of data for training ,and it may be more accurate when we use the full training set.  

- Different randomization.With certain algorithms,random numbers are involved ,but we're not using any of these.   



# Generating better predictions for your Kaggle submission

The forums are very important in Kaggle, and can often help you find nuggets of information that will let you boost your score. The Expedia competition is no exception. There's a post that details a data leak which allows you to match users in the training set from the testing set using a set of columns including `user_location_country` ,and `user_location_region`    

We'll use the information from the post to match users from the testing set back to the training set, which will boost our score.Based on the forum thread,its okay to do this,and the competition wont be updates as a result of the leak

# Finding matching users
The first step is to find users in the training set that match users in the testing set.   

In order to do this,we need to:  

- split the training data into groups based on the match columns  
- Loop through the testing data  
- Create an index based on the match columns  
- Get any matches betweeb the testing data and the training data using the groups.  

Here's the code to accomplish this:   

In [62]:
match_cols = ['user_location_country' , 'user_location_region', 
              'user_location_city','hotel_market','orig_destination_distance']

#groups =t1.groupby(match_cols)  
groups =train.groupby(match_cols) 

def generate_exact_matches(row,match_cols):
    index = tuple([row[t] for t in match_cols])
    try:
        group = groups.get_group(index)
    except Exception:
        return []
    clus = list(set(group.hotel_cluster))
    return clus

exact_matches = []
for i in range(test.shape[0]):
    exact_matches.append(generate_exact_matches(test.iloc[i], match_cols))
    

At the end of this loop, we'll have a list of lists that contain any exact matches between the training and the testing sets. However there aren't that many matches.To accurately evaluate error,we'll have to combine these predictions with our earlier predictions.Otherwise , we'll get a very low accuracy value,because most rows have empty lists for predictions.  

# Combining predictions  

We can combine different lists of predictions to boost accuracy .Doing so will also hekp us see how good our eaxct match strategy is.To do this,we'll have to:  

- Combine `exact_matches`,`preds`, and `most_common_clusters`  
- Only take the unique predictions,in sequential order,using the `f5` function 
- Ensure we have a maximum of `5` predictions for each row in the testing set.  

Here's how we can do it : 

In [63]:
def f5(seq, idfun=None): 
    if idfun is None:
        def idfun(x): return x
    seen = {}
    result = []
    for item in seq:
        marker = idfun(item)
        if marker in seen: continue
        seen[marker] = 1
        result.append(item)
    return result
    
full_preds = [f5(exact_matches[p] + preds[p] + most_common_clusters)[:5] for p in range(len(preds))]
#metrics.mapk([[l] for l in t2["hotel_cluster"]], full_preds, k=5)


This is looking quite good in terms of error - we improved dramatically from earlier ! We could keep going and making small improvements,but we're probably ready to submit now.

# Making a Kaggle submission file

Luckily,because of the way we wrote the code,all we have to do submit is assign `train` to the variable `t1` and `test` to the variable `t2`.Then ,we jsut have re-run the code to make predictions.Re- running the code over the `train` and `test` sets should take less than an hour.  

Once we have predictions,we just have to write them to a file:

In [66]:
write_p = [" ".join([str(l) for l in p]) for p in full_preds]
write_frame = ["{0},{1}".format(test["id"][i], write_p[i]) for i in range(len(full_preds))]
write_frame = ["id,hotel_cluster"] + write_frame
with open("predictions.csv", "w+") as f:
    f.write("\n".join(write_frame)) 
    

# Future techniques to try

Once you have a stable foundation on which to run your code,there are a few avenues to explore in terms of techniques to boost accuracy:  

- Finding similarity between users,then adjusting hotel cluster scores based on similarity  
- Using similarity between destinations to group multiple destinations together  
- Applying machine learning within subsets of the date
- Combining different prediction strategies in a less naieve way
- Exploring the link between hotel clusters and regions more.  